In [1]:
!pip install -q transformers accelerate bitsandbytes einops
!pip install -q pymupdf


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=4096,
    do_sample=False,
    return_full_text=False
)


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

...

model-00018-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/4.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [27]:
def preprocess_cv_text(text):
    import re
    text = re.sub(r'\n+', '\n', text)
    text = re.sub(r'[ \t]+', ' ', text)
    return text.strip()


In [28]:
import fitz

def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text= "\n".join(page.get_text() for page in doc).strip()
    return preprocess_cv_text(text)


In [8]:
def build_prompt(cv_text):
    return f"""
Tu es un extracteur d'informations structuré. À partir du texte du CV ci-dessous, fournis un objet JSON valide contenant les champs suivants :

- name
- profile
- phone
- email
- address
- experience (liste de: company, title, start_date, end_date, description)
- education (liste de: university, degree, start_date, end_date, description)
- certifications (liste de: title, organization, date, description)
- skills (liste de chaînes)

Donne uniquement un JSON valide avec tout le text du CV n’oublie aucun mot, sans commentaire ni texte en plus.

CV:
\"\"\"{cv_text}\"\"\"
"""


In [9]:

pdf_path = "CV_Achraf_HT_Directeur et Chef de projet IT_2025.pdf"

cv_text = extract_text_from_pdf(pdf_path)
prompt = build_prompt(cv_text)

output = pipe(prompt)[0]["generated_text"]
print(output)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



```json
{
  "name": "Achraf HADJ TAIEB",
  "profile": "Directeur IT et Chef de projet expérimenté, certifié Agile Scrum PSPO I, avec plus de 16 ans d’expérience dans la gestion de projets stratégiques et la conduite de transformations digitales.",
  "phone": "+33 6 44 18 07 72",
  "email": "achraf.ht@gmail.com",
  "address": {
    "location": "Paris",
    "carbonne_impact": "9.78 g CO₂e"
  },
  "experience": [
    {
      "company": "Allianz Trade",
      "title": "Directeur et Chef de projet IT",
      "start_date": "Nov 2021",
      "end_date": "Dec 2024",
      "description": "Programme Qirin - Document Services"
    },
    {
      "company": "AXA Assurance",
      "title": "Digitalisation des processus RH et pilotage de projets transverses",
      "start_date": "",
      "end_date": "",
      "description": "Sous La direction du Head of IT"
    },
    {
      "company": "Ville de Bruxelles",
      "title": "Transformation digitale et gestion d’applications critiques en architectur

In [30]:
pope = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=8192,  
    do_sample=False,
    return_full_text=False
)

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [36]:
def build_prompt_position(cv_text):
    return f"""
Tu es un assistant chargé d’analyser un texte brut de CV. Je veux que tu extraies, pour chaque champ suivant, les positions de début et de fin (offsets) dans le texte d’origine. Ne donne pas la valeur extraite, seulement les positions en JSON.
Champs à extraire :
name (nom complet de la personne)
profile (résumé ou accroche professionnelle)
phone
email
address
Ensuite, pour les sections structurées, retourne une liste de blocs, chacun avec les positions des sous-champs suivants :
experience: liste de blocs contenant :
company
title
start_date
end_date
description
education: liste de blocs contenant :
establishment
degree
graduation_date
description
certifications: liste de blocs contenant :
title
organization
date
description
skills: liste de chaînes avec leurs positions (chaque compétence doit être localisée avec sa position de début et de fin)
Parcours tout le text et donne uniquement un JSON valide avec le format JSON des positions de début et de fin (offsets) des champs dans le texte d’origine sans commentaire ni texte en plus.

CV:
\"\"\"{cv_text}\"\"\"
"""


In [40]:

pdf_path = "CV_Achraf_HT_Directeur et Chef de projet IT_2025.pdf"


cv_text1 = extract_text_from_pdf(pdf_path)
prompt = build_prompt_position(cv_text)

output = pope(prompt)[0]["generated_text"]
print(output)


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


here
Présentation 
 
Directeur IT et Chef de projet expérimenté, certifié Agile Scrum PSPO I, avec plus de 16 ans d’expérience dans la 
gestion de projets stratégiques et la conduite de transformations digitales. Reconnu pour ma rigueur, ma capacité à 
piloter des équipes pluridisciplinaires et multiculturelles, j’interviens avec efficacité sur des environnements 
complexes, en garantissant un haut niveau de qualité, de performance et de conformité. 
 
Information 
Localisation : Paris - Impact Carbonne : 9.78 g CO₂e 
 
TJM : 700€ 
Disponibilité : Immédiate 
Compétences clés 
Managériales 
§ 
Leadership avéré dans la direction de projets 
complexes (jusqu’à 40 pays impactés). 
§ 
Gestion de programmes stratégiques et 
coordination d'équipes pluridisciplinaires. 
§ 
Organisation et animation de comités de 
gouvernance (architecture, stratégie IT). 
§ 
Capacité à définir et piloter des stratégies de 
transformation (digitalisation, migration, 
innovation). 
§ 
 
Chefferie/Directions de p

KeyboardInterrupt: 